In [1]:
# 采集1min kline data

In [2]:
import pandas as pd
import numpy as np
import jqfactor_analyzer as ja
import numba
import yfinance as yf
import backtrader as bk
import akshare as ak
import os
import scipy
import statsmodels.api as sm
import random
import math

In [3]:
import math
s1 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0',"AU0", "AG0", "CU0", "AL0", "ZN0", "PB0", "NI0", "SN0", "RB0", "HC0",
    "BU0", "RU0", "FU0", "SP0", "WR0", "SS0", "SA0", "UR0", "SR0", "CF0",
    "TA0", "MA0", "FG0", "RS0", "OI0",'AU0','AG0'
    "JR0", "SF0", "SM0", "AP0", "CJ0", "CY0", "RR0", "BB0", "FB0",
    "EB0", "V0", "L0", "PP0", "CS0", "JD0", "BU0", "HC0"]
#s2 = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0']

#内盘
start_date = '2021-01-01'
end_date = '2023-08-10'

#@numba.jit(nopython=True)
def future_AA_arb():
    for i in range(1000000):
        futures_foreign_hist_df = []
        futures_zh_daily_sina_df = []
        # bc-cu,jm-j,rb-hc,cf-cy(x),wr-rb
        if i % 100 == 1:
            print(i)
        s11 = random.choice(s1)
        if s11 != 'C0':
            continue
        try:
            futures_foreign_hist_df = ak.futures_zh_daily_sina(symbol=s11)
        except:
            continue
        futures_foreign_hist_df = futures_foreign_hist_df[(futures_foreign_hist_df.date >= start_date) & (futures_foreign_hist_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_foreign_hist_df["date"].min(), end=futures_foreign_hist_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_foreign_hist_df['date'] = pd.to_datetime(futures_foreign_hist_df['date'])
        futures_foreign_hist_df = futures_foreign_hist_df.set_index("date")
        futures_foreign_hist_df = futures_foreign_hist_df.resample("D").ffill()
        #print(futures_foreign_hist_df['close'])
        #print(futures_foreign_hist_df)
    
        #
        s22 = random.choice(s1)
        if s22 == s11:
            continue
        try:
            futures_zh_daily_sina_df = ak.futures_zh_daily_sina(symbol=s22)
        except:
            continue
        futures_zh_daily_sina_df = futures_zh_daily_sina_df[(futures_zh_daily_sina_df.date>=start_date) & (futures_zh_daily_sina_df.date <= '2023-08-10')]
        date_range = pd.date_range(start=futures_zh_daily_sina_df["date"].min(), end=futures_zh_daily_sina_df["date"].max(), freq="D")  # freq="D"表示按天，可以按分钟，月，季度，年等
        futures_zh_daily_sina_df['date'] = pd.to_datetime(futures_zh_daily_sina_df['date'])
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.set_index("date")
        futures_zh_daily_sina_df = futures_zh_daily_sina_df.resample("D").ffill()
        
        #print(futures_zh_daily_sina_df)
        # print(s1[i])
        # print(len(futures_foreign_hist_df))
        # print(len(futures_zh_daily_sina_df))
        #print(np.array(futures_foreign_hist_df['close']))
        ## 计算Pearson相关系数

        #shift 1
        df_for = [futures_foreign_hist_df['close'][0] for i in range(1)] + futures_foreign_hist_df['close'][:-1].tolist()
        
        # pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'][10:], df_for[10:])
        # print("shifted %s-%s Pearson Correlation:" %(s1[i],s2[i]), pearson_corr)
        try:
            pearson_corr, _ = scipy.stats.pearsonr(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
        except:
            continue
        if abs(pearson_corr) > 0.5:
            print("%s-%s Pearson Correlation:" %(s11,s22), pearson_corr)
                 # 进行ADF检验
            try:
                # result = sm.tsa.coint(futures_zh_daily_sina_df['close'], futures_foreign_hist_df['close'])
                # cointegration_statistic = result[0]
                # p_value = result[1]
                
                # print("cointegration_statistic:", cointegration_statistic)
                # print("p-value:", p_value,'\n')
                print("sgima",np.std(futures_foreign_hist_df['close']-futures_zh_daily_sina_df['close']))
            except:
                continue
#future_AA_arb()

In [28]:
start_year = 2301
end_year = 2312

year_strings = [str(year) for year in range(start_year, end_year + 1)]
print(year_strings)

symbols = ['SC0','PG0','AU0','AG0','CU0','PB0','NI0','BC0','A0','B0','M0','JM0','J0','C0','CS0','MA0','PP0','CU0','RU0','Y0','P0',"AU0", "AG0", "CU0", "AL0", "ZN0", "PB0", "NI0", "SN0", "RB0", "HC0",
    "BU0", "RU0", "FU0", "SP0", "WR0", "SS0", "SA0", "UR0", "SR0", "CF0",
    "TA0", "MA0", "FG0", "RS0", "OI0",'AU0','AG0'
    "JR0", "SF0", "SM0", "AP0", "CJ0", "CY0", "RR0", "BB0", "FB0",
    "EB0", "V0", "L0", "PP0", "CS0", "JD0", "BU0", "HC0"]

symbols = [s[:-1] for s in symbols]
for s in symbols:
    print(s)
    for i in year_strings:
        csv_filename = './'+s+'_1min.csv'
        try:
            futures_zh_minute_sina_df = ak.futures_zh_minute_sina(symbol=s+i, period="1")
            futures_zh_minute_sina_df.to_csv(csv_filename, mode='a')
        except:
            continue

['2301', '2302', '2303', '2304', '2305', '2306', '2307', '2308', '2309', '2310', '2311', '2312']
AU
AG
CU
PB
NI
BC
A
B
M
JM
J
C
CS
MA
PP
CU
RU
Y
P
AU
AG
CU
AL
ZN
PB
NI
SN
RB
HC
BU
RU
FU
SP
WR
SS
SA
UR
SR
CF
TA
MA
FG
RS
OI
AU
AG0JR
SF
SM
AP
CJ
CY
RR
BB
FB
EB
V
L
PP
CS
JD
BU
HC
